# Lab 2: Large-scale atmospheric circulation


## Overview 

This lab explores general atmospheric circulation patterns using the ERA5 atmospheric reanalysis. ERA5 is produced by the Copernicus Climate Change Service at the European Centre for Medium-Range Weather Forecasts (ECMWF). The reanalysis combines model data with observations from around the world to provide hourly estimates of a large number of atmospheric, land, and oceanic climate variables. The data was accessed from the [ECMWF Climate Data Portal](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5).

The full reanalysis product covers the period from January 1940 to present. The original monthly datasets are available at 0.25$^\circ$ latitude x 0.25$^\circ$ longitude horizontal resolution. This yields 4.2 GB of data *per atmospheric variable*, so the files were regridded to a much coarser  ~4 $^\circ$ latitude x 4$^\circ$ longitude horizontal resolution (16.7 MB per file). The 4$^\circ$ resolution will still show the broad global patterns. However, if you would like to work with the higher-resolution files please ask Dr. Lawman and she will share them with you.

You will analyze global monthly and seasonal variations in 2-meter (near-surface) temperature, precipitation, and wind speed. You will then calculate anomalies and explore the spatial patterns of rising global temperatures.  All code and figures will be generated in this Jupyter Notebook using in Python. Please use comments (`#`) to organize your code and make it more readable.  **This lab is worth 60 points.**

**Required data:** (Download from GitHub)
- **2 m temperature:** ERA5_monthly_t2m_regrid.nc 
- **Total precipitation:** ERA5_monthly_tp_regrid.nc
- **10 m wind speed:** ERA5_monthly_si10_regrid.nc
- **10 m u-component of wind (zonal wind):** ERA5_monthly_u10_regrid.nc
- **10 m v-component of wind (meridional wind):** ERA5_monthly_v10_regrid.nc

**In this lab you will learn and practice the following skills:**

- Interpreting global atmospheric patterns
- Geospatial analysis 
- Statistical analysis
- Manipulating gridded data with Xarray
- Data visualization with Cartopy and Matplotlib

**Before you begin:**

You will need to install an additional Python library called NetCDF4. The steps are outlined below: 
1. Open a new terminal (Mac) or command prompt (Windows)
2. Activate your Atmo_EV333 environment: `conda activate Atmo_EV333`
3. Install NetCDF4 by typing in the command line: `pip install netCDF4`
4. Now you are ready to proceed with the rest of the lab

**To submit via Canvas:**

To foster a collaborative learning environment, you are encouraged to work in groups of 2-3, but each person must write and submit their own code, and individually answer all interpretation questions. All students must complete all problems - it is against the honor code to divide the problems up among different individuals. 

- Prior to submitting your lab assignments on Canvas, please name your Notebook files using the following syntax: *LastName_FirstName_EV333_Lab2.ipynb.*
- Type you responses to the short-answer questions in this Notebook. Add a cell and select 'Markdown' instead of 'Code' from the drop down menu. Add **Answer:** and provide your written responses.
- Re-run all cells to ensure that the Notebook runs completely through without errors and that all figures are displayed.
- Upload your final Jupyter Notebook (.ipynb file) to Canvas

---
## ERA5 Variable Descriptions
*Descriptions retrieved from the [Copernicus Climate Data Portal](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=overview).*

**2m temperature (K)**: This parameter is the temperature of air at 2m above the surface of land, sea or inland waters. 2m temperature is calculated by interpolating between the lowest model level and the Earth's surface, taking account of the atmospheric conditions. This parameter has units of kelvin (K). Temperature measured in kelvin can be converted to degrees Celsius (°C) by subtracting 273.15.

**Total precipitation (m)**: This parameter is the accumulated liquid and frozen water, comprising rain and snow, that falls to the Earth's surface. It is the sum of large-scale precipitation and convective precipitation. Large-scale precipitation is generated by the cloud scheme in the ECMWF Integrated Forecasting System (IFS). The cloud scheme represents the formation and dissipation of clouds and large-scale precipitation due to changes in atmospheric quantities (such as pressure, temperature and moisture) predicted directly by the IFS at spatial scales of the grid box or larger. Convective precipitation is generated by the convection scheme in the IFS, which represents convection at spatial scales smaller than the grid box. This parameter does not include fog, dew or the precipitation that evaporates in the atmosphere before it lands at the surface of the Earth. This parameter is accumulated over a particular time period which depends on the data extracted. For the monthly averaged reanalysis, the accumulation period is 1 day.

**10m u-component of wind (m/s):** This parameter is the eastward component of the 10m wind. It is the horizontal speed of air moving towards the east, at a height of ten metres above the surface of the Earth, in metres per second. Care should be taken when comparing this parameter with observations, because wind observations vary on small space and time scales and are affected by the local terrain, vegetation and buildings that are represented only on average in the ECMWF Integrated Forecasting System (IFS).

**10m v-component of wind (m/s)**: This parameter is the northward component of the 10m wind. It is the horizontal speed of air moving towards the north, at a height of ten metres above the surface of the Earth, in metres per second. Care should be taken when comparing this parameter with observations, because wind observations vary on small space and time scales and are affected by the local terrain, vegetation and buildings that are represented only on average in the ECMWF Integrated Forecasting System (IFS).

**10m wind speed (m/s):** This parameter is the horizontal speed of the wind, or movement of air, at a height of ten metres above the surface of the Earth. The units of this parameter are metres per second. Care should be taken when comparing this parameter with observations, because wind observations vary on small space and time scales and are affected by the local terrain, vegetation and buildings that are represented only on average in the ECMWF Integrated Forecasting System (IFS). The eastward and northward components of the horizontal wind at 10m are also available as parameters (descriptions above).

---

In [1]:
# import Python packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

## Part 1: Generate a  global map of mean surface temperature from 1940-2022

First we need to import the ERA5 2 m surface temperature dataset. The dataset is in the NetCDF (NetCDF (network Common Data Form) format and has the extension *.nc*. NetCDF data is a file format for storing multidimensional scientific data. In this lab you will use the Python [Xarray](https://docs.xarray.dev/en/stable/) library, which is very good at working with netCDF data. You already installed xarray when setting up your Python environment for this class. 

Xarray was imported as `xr` in the above cell. We will first use the [open_dataset](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html) function to import the data. open_dataset has a lot of optional arguments but for this we only care about the first argument which is the file name.

### Step 1: Load the 2m temperature data:

**<span style='color:Red'> Insert a cell below using the `+` icon.  </span> Add the following code:**

```
# import the 2 m surface temperature data set using xarray 
ds = xr.open_dataset('./Data/ERA5_monthly_t2m_regrid.nc')

```

You will need to change `Data/ERA5_monthly_t2m_regrid.nc` if you are storing the netCDF file in another directory on your computer. If this is the case, put the full path for the file in single quotes. 

The return of open_dataset is a [dataset object](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset), here called `ds`. We can confirm this by using the type function. Identify and print the the type of your dataset using `print(type(ds))`. 

Now that we have the dataset imported we need to understand what is in it. We can do this by printing it.

In [3]:
# display ds to see what is contained in the file
ds

<xarray.Dataset>
Dimensions:  (time: 1005, lon: 90, lat: 46)
Coordinates:
  * time     (time) datetime64[ns] 1940-01-01 1940-02-01 ... 2023-09-01
  * lon      (lon) float64 1.875 5.875 9.875 13.88 ... 345.9 349.9 353.9 357.9
  * lat      (lat) float64 88.12 84.12 80.12 76.12 ... -83.88 -87.88 -90.0
Data variables:
    t2m      (time, lat, lon) float32 ...

Notice that there are 3 main categories of information **Dimensions**, **Coordinates**, and **Data variables**. All 3 are related in the following ways:
- Dimmensions are the number of elements that make up something
- Coordinates are the dependent variables (where the number of elements is equal to its dimmension)
- Data variables are the independent variables we  care about (where the number of elements is equal to the product of all its dimmension)

**See this link for an illustration of how the DataSet is organized:** https://docs.xarray.dev/en/latest/_images/dataset-diagram.png.

You can click on the Show/Hide attributes icon on the right side of the displayed DataSet (looks like a page icon) to learn more about the coordinates, data variables, and their units.

For furthur disambiguation of terminology, see the [xarray terminology guide](https://docs.xarray.dev/en/latest/user-guide/terminology.html) linked here. 

For this lab we want to find the mean 2m temperature (abbreviated `t2m`). The only data variable in our dataset
is `t2m` so it is clear what we will be working with this. We will want to calculate a mean of 1940-2022 so let's look at `time` in our dataset. We can access `time` by treating our DataSet like a dictionary: `ds['time']`.

**<span style='color:Red'> Insert a cell below (`+`) </span> and add the following code to display `time`:**
```
print(ds['time'])
```
**[1 pt]** What time period does the data set cover? What is the temporal resolution of the data? What is the data type? Answer these questions by qualitatively viewing the `time` coordinate and printing `type(ds['time'])`.

### Step 2: Subset the data to January 1940- December 2022

Notice that we are now working with a DataArray instead of a DataSet. Both act very similar but the key differece
is that a DataSet is a collection of 1 or more DataArrays. Looking at the time values we can see that our data
starts in January of 1940 and ends in September of 2023. We will subset the data to exclude 2023 so that the final time is December 2022.

**[2pts]<span style='color:Red'> Insert a cell below (`+`) </span>** and subset the data to January 1940-December 2022. You will want to use the `.isel()` method and select all time less than 2023. `ds.isel(time=(ds.time.dt.year < 2023))`. Print the DataSet to make sure you correctly subset it.

*For more information on subsetting DataSets see:*
- https://docs.xarray.dev/en/latest/user-guide/indexing.html
- https://docs.xarray.dev/en/latest/user-guide/time-series.html

### Step 3: Calculate the long-term mean over all time

Now that our data ends in December of 2022, let's use the [mean](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.mean.html) method to average our data over all time. 

**<span style='color:Red'> Insert a cell below using the `+` icon </span>** and calculate the mean over all time (across the time dimension). Define this as a variable `m`. **[0.5pt] What are the dimensions of the mean?** Some example code is provided below:

```
# calculate the mean over all time
m = ds.mean(dim='time')
print(m)
```

**[0.5pt] What are the units of the temperature values?** For this lab we want to work in $^\circ$C. **<span style='color:Red'> **[1.5pts]** Insert a cell below using the `+` icon </span>** and convert the mean temperature values to $^\circ$C. You will need to apply the conversion specifically to the `t2m` DataArray by accessing it like a dictionary in square brackets `m['t2m']`. Define a variable called `m_t2m` that has the DataArray with temperature in degrees C. 

Print your mean DataArray and check the values to confirm that the calculation was performed correctly.

We are almost ready to plot the data. But first, find the min and max values so that you have a starting point for the levels of the colormap. 

**[1pt] <span style='color:Red'> Insert a cell below (`+`) </span>** and calculate the minimum and maximum mean values using Python functions. **What are the maximum and minimum values (and their units)? Print the values.** 

*Hint: Use the `.max()` and `.min()` methods. `.max().values` will print the values of the DataArray*

### Step 4: Generate a global map of 2 m air temperature

Now let's make a map of the data! You will need to specify a colormap from the [cmocean: colormaps for oceanography](https://matplotlib.org/cmocean/) Python package this is installed in your Python environment and imported in thius Jupyter Notebook. Visit the cmocean documentation linked above. **[0.5pt] Which colormap do you think would be most appropriate for 2 m temperature? Why?**

**<span style='color:Red'> Insert a cell below (`+`) </span>:** 
1. **[0.5pt]** Define your colormap as a variable called `cmap.` Some example code is provided below, but you will want to change the colormap. 

```
# selected color map from cmocean colormaps for oceanography (update with a color map for temperature)
cmap = cmocean.cm.Rain
```

2. **[0.5pt]** Define a range of values for the colorbar, based on the minimum and maximum values calculated above. 

*Hint: Define a variable called `lev` as a NumPy array spanning from the minimum to the maximum values (rounded to the nearest integer). NumPy documentation to help you define evenly spaced values within a given interval is linked [here](https://numpy.org/doc/stable/reference/generated/numpy.arange.html).*

```
# NumPy array for the color bar levles, here from -50 degC to +30 degC, in increments of 2 degC 
lev = ...add your code here...
```

3. **[0.5pt]** Define the map projection. The Python package Cartopy has many [map projections](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html). **In this lab we will use the Robinson projection to make global maps centered on 180$^\circ$ longitude (in the middle of the Pacific Ocean).** To define the map projection as a variable called `proj` and specify a central longitude of 180 degrees:

```
# map projection
proj = ccrs.Robinson(central_longitude=180)
```

Making maps takes some practice, so example code has been provided below for you as a guide. **Please carefully go through the code line by line to make sure you understand each line before proceeding.**

**[2pts] <span style='color:Red'> Insert a cell below (`+`) </span>** and add the following code to generate a map of mean 2 m air temperature. 


```
# define figure and axes, figure size, and resolution (300 dpi))
fig = plt.figure(figsize=(9, 4.5), dpi=300)
ax = plt.axes(projection = proj)

# filled contour map of mean temperature
m_t2m.plot.contourf(
    x = 'lon',
    y = 'lat',
    ax=ax,
    transform=ccrs.PlateCarree(),
    levels=lev,
    extend='both',
    colors=cmap,
    add_colorbar=True)

# add coastlines
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.coastlines(
    resolution='110m')  #Currently can be one of “110m”, “50m”, and “10m”.

# add grid lines
gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  linewidth=1,
                  color='gray',
                  alpha=0.5,
                  linestyle='--')

# add title
ax.set_title("Mean Temperature")

# save figure 
fig.savefig('ERA5_mean_t2m.png', facecolor = 'white', transparent = False, bbox_inches ='tight')
```

**The code above:**

- Defines the figure, axes, and figure resolution (300 dpi selected so the final map isn't blurry)
- Generates a filled contour plot using `m_t2m.plot.contourf()` (where `m_t2m` is your DataFrame with 2 m air temperature in degrees C). The label for the color bar is specified by the colorbar keyword arguments `cbar_kwargs = {"label":"Temperature ($^\circ C$)"}`
- Sets the levels for the colorbar (`lev` variable) and the colormap (`cmap` variable)
- Adds coastlines using the Cartopy `BORDERS` feature (imported in this Jupyter Notebook). Since this is a global map, the coarser 110m resolution is selected.
- Adds latitude and longitude grid lines (as gray dashed lines)
- Adds a descriptive title and labels the colorbar with the appropriate units
- Saves the figure as `Lab2_ERA5_mean_t2m.png`


**<span style='color:Red'> Insert a cell below (`+`) </span>** and paste the following code to generate a global map of mean 2 m air temperature.

## Part 2: Generate a global map of mean precipitation from 1940-2022

**Using the 2 meter air temperature code as a guide**, generate a global map of mean precipitation for 1940-2022. Broadly, you will take the following steps:
- Import the total precipitation file using xarray (the variable name is `tp`)
- Subset the `tp` DataSet to January 1940 to December 2022
- Calculate the overall mean across the time dimension
- Convert total precipitation from units of m/day to mm/day (*Note: the units of the original dataset are in meters. For this parameter, the accumulation period is 1 day. Your final units will be mm/day*)
- Define a suitable colormap (`cmap`) that is appropriate for precipitation data
- Define a suitable colormap range (`lev`) based on the minimum and maximum of the data
- Generate a filled contour map using `.contourf()`
- Update the colorbar label with the associated units
- Add coastlines and latitude/longitude gridlines
- Add a title
- Save the figure as `Lab2_ERA5_mean_tp.png`

**<span style='color:Red'> [8pts] Insert cell(s) below (`+`) </span> to generate a global map of mean precipitation.** To receive full credit, your map must include all the elements listed above. Add `#` throughout to help keep the code organized. Save the figure as `Lab2_ERA5_mean_tp.png`.

## Part 3: Generate a global map of mean wind speed with vectors showing the direction of the mean winds (for 1940-2022)

Global wind patterns are very important for global precipitation and precipitation - evaporation patterns! Now let's generate a global filled contour map of average wind speed (variable `si10`). The process of making the wind speed map will be nearly the same as 2 meter air temperature and total precipitation.

Adding the wind vectors is a little more tricky, so example code is provided below to help guide the process. Use `DataSet.plot.quiver()` to overlay the wind vectors. To do this, you will need to load both the 10 m u-component of the wind (zonal wind), and the 10 m v-component of the wind (meridional wind) and combine them by their coordinates. This will combine the u and v wind component data into one by using their dimension coordinates.

The documentation for Xarray's quiver method is linked [here](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.plot.quiver.html). The inputs for `.quiver()` are x (longitude), y (latitude), u (u component), and v (v component). Set the `regrid_shape` argument so that the arrows are not too close together. Example code that you can copy/paste and modify is provided below:

```
# load the u and v wind components
u = xr.open_dataset(...add your file here...)
v = xr.open_dataset(...add your file here...)

# calculate the mean across the time dimension for both u and v wind components
...add your code here...

# combine by coordinates. This combines the given datasets (or data arrays) into one by using dimension coordinates
ds = xr.combine_by_coords([u,v])

# add quivers using ds.plot.quiver(); specify x as longitude, y as latitude, and u and v as the 10 m u and v wind components, respectively. Set regrid_shape=20 to avoid having quivers that are too close together.
ds.plot.quiver(
    x = 'lon',
    y = 'lat',
    u = 'u10',
    v = 'v10',
    ax=ax,
    transform=ccrs.PlateCarree(),
    extend='both',
    regrid_shape=20)

```

**<span style='color:Red'> [12pts] Insert cell(s) below (`+`) </span>** to load the wind data and generate a map of mean annual wind speed. 6pts for the filled contour map with wind speed and 6pts for adding the wind vectors. 

Please choose a colormap, set the color bar levels, add a title, coastlines, and latitude/longitude gridlines as you did for 2 m air temperature and precipitation. Save your figure as: `Lab2_ERA5_mean_wind.png`

## Part 4: Generate global seasonal mean precipitation maps (for 1940-2022)

How does precipitation vary throughout the year? To find this out, let's make maps of seasonal mean precipitation. Seasonal averages are for December-January-February (DJF), March-April-May (MAM), June-July-August (JJA), and September-October-November (SON).

To calculate a seasonal mean, use the `groupby.()` method you used in Lab 1, but instead here on DataSets and DataArrays instead of Pandas DataFrames. Here you will group the data by season: `ds.groupby(ds['time'].dt.season)` and calculate the mean using the `.mean()` method. For example: `ds_seas = ds.groupby(ds['time'].dt.season).mean()` where `ds_seas` is a variable for the seasonal averages.

`ds_seas[0]` will be the DJF average, `ds_seas[1]` will be the JJA average. You may want to define variables to help keep the seasonal means organized. For example:

```
# define seasonal means by subsetting ds_seas
DJF = ds_seas[0]
JJA = ds_seas[1]
MAM = ...add your code...
SON = ...add your code...
```

*Remember the precipitation will need to be converted from m/day to mm/day.*

**<span style='color:Red'> [10pts, 2.5pts per plot] Insert cell(s) below (`+`) </span>** to load the precipitation data again and calculate seasonal averages. Generate 4 figures that have the DJF, MAM, JJA, and SON averages. Use consistent formatting for all figures (e.g., colorbars, colorbar levels, titles, etc.) Save your figure as `Lab2_ERA5_mean_tp_seasonal_SEASON.png` where SEASON is DJF, JJA, MAM, SON (e.g., for DJF: `Lab2_ERA5_mean_tp_seasonal_DJF.png`)

## Part 5: Generate temperature anomaly maps

In the last section of this lab, you will create 2 m air temperature anomaly maps showing the difference (i.e., temperature anomalies) between two 20-year intervals. Here you will plot the difference between (2003-2022) and (1961-1980). To select a temporal subset, use the `time=slice() method`. An example is provided below:

```
# load the 2 m air temperature data
ds = xr.open_dataset('./Data/ERA5_monthly_t2m_regrid.nc')

# subset the data to 2003-01 to 2022-12
ds1 = ds.sel(time=slice('2003-01-01', '2022-12-01')).mean(dim='time')

# extract the t2m dataset for the selected subset
ds1 = ds1['t2m']
```

**<span style='color:Red'> [8pts] Insert cell(s) below (`+`) </span>** to:
1. Load the 2 meter temperature NetCDF file 
2. Subset the data to (2003-2022) and the (1961-1980)reference interval
3. Calculate the mean for each interval
4. Calculate the anomalies (the difference). For example `anom = ds2 - ds1` where `ds1` and `ds2` are the DataArrays for the different time intervals. Calculate the (2003-2022) - (1961-1980) anomalies.
5. Generate a difference map. The [cmocean Balance](https://matplotlib.org/cmocean/) colormap is recommended. Blue colors will indicate cool anomalies and red colors will indicate warm anomalies. 
6. Add titles, coast lines, grid lines, etc. as you did with the other global maps.
7. Save the figure as `Lab2_ERA5_anom_t2m.png`

## Part 6: Global atmospheric circulation synthesis questions

**[12pts total] All questions are 1 pt each.**

1. Where are the highest surface temperatures found? Where are the lowest temperatures found? Do the regions seem reasonable to you based on your knowledge of the Earth? Explain.

2. Please select a region on the temperature map that looks interesting to you. Describe why you find this region interesting and explain what contributes to the average temperature in that region. 

3. Why are the temperature patterns not zonally symmetric?

4. What is the range of wind speeds that occur near the surface? Please include the units. 

5. What latitude range are the winds the fastest? Based on the three-cell model of atmospheric circulation, what cell are the fastest winds in? For this question you can ignore small patches of high wind speeds and focus on the broader global patterns. 

6. What latitude are the winds the slowest? Again, for this question you can ignore small patches of low wind speeds and focus on the broader global patterns.

7. Let’s focus on the tropics (30°S – 30°N). On average, what direction do the winds blow from in the Northern Hemisphere between 0-30°N? What direction do the winds blow from in the Southern Hemisphere between 0-30°S? What do we call these winds?

8. What happens to the winds near the equator? Are they converging or diverging? 

9. What part of the world has the highest precipitation? In this lab, please focus on broad bands of precipitation. Do you see a connection between the surface winds and the amount of precipitation? Does this band of precipitation have a special name?

10. How do large-scale wind patterns influence rainfall patterns? 

11. How do bands of precipitation change seasonally throughout the year?

12. Explain the pattern of warming observed during the past 20-years (2003-2022) compared to the 1961-1980 reference interval. Which areas are warming the most? Why?


**<span style='color:Red'> Insert a cell below (`+`) </span>** to answer the synthesis questions. Select Markdown instead of Code from the drop-down menu and type your responses. Each question is worth 1 point. 